In [1]:
import pandas as pd
import numpy as np
import math
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from IPython.core.display import display

In [3]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder


In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [5]:
import category_encoders as ce


In [6]:
SEED = 42
target = 'price'

In [7]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [8]:
""" function to re-orgonize list to dataframe"""
def list_cut(to_cut):
    """
    function to re-organize list to dataframe
    :param to_cut: list to re-order
    :return: dataframe
    """

    # call lambda for further cut and for new names
    # Объявим функции для разбиение списка на список стеков и генерировнаия названий для датафрейма
    to_cut = list(np.sort(to_cut))
    cut_list = lambda item: [item[i:i+7] for i in range(0, len(item), 7)]
    name_list = lambda nom: ["vectors"+ str(i) for i in range(0,math.ceil(nom/7))]

    # making dataframe
    # создание датафрейма и присовение названий векторам (пропуски заполняем '')
    df_ = pd.DataFrame(cut_list(to_cut)).T
    df_.columns = name_list(len(to_cut))
    df_.fillna('', inplace=True)

    # display it
    # выводим на экран
    display(df_)

In [9]:
""" Функция проверки корреляции с целевым вектором в виде графика heatmap """
def heatmap(drop_list, dd, target = target):
    """
    function for exploring target correlation with hitmap plot
    :param drop_list: list to delete from analysis before
    :param dd: dataset to analyse
    :param target: target vector,
    :return: table with excluded vectors and hitmap plot of correlations
    """
    dlst = list(set(dd.columns) - set(drop_list) - {target})
    dlst = list(np.sort(dlst))

    dlst.append(target)
    sns.set(font_scale=1)
    plt.subplots(figsize=(16, 16))
    sns.heatmap(dd[dlst].corr(), square=True,
                annot=True, fmt=".3f", linewidths=0.1, cmap="RdBu", vmin=-1, vmax=1)

    # call function to cul drop-list for table.
    list_cut(drop_list)

In [10]:
path = '/Users/irenaradzevich/Documents/sergei_dev/ project_05/'

In [11]:
auto = pd.read_csv(path+'all_auto_ru_09_09_2020.csv')

In [12]:
add = pd.read_csv(path+'train2.csv')

In [13]:
test = pd.read_csv(path+'test.csv')
train = pd.read_csv(path+'train.csv')
print(f'Train data shape : {train.shape},\n  add data shape : {add.shape},\n auto data shape : {auto.shape},\n test data shape : {test.shape}')

Train data shape : (103794, 33),
  add data shape : (25465, 33),
 auto data shape : (89378, 26),
 test data shape : (34686, 32)


---
# 0. Crearing data
---

In [14]:
# append addon data to train dataset
train = train.append(add)

# drop duplicates and records with zero-prices and records on new cars (they has different webpage structure)
train.drop_duplicates(inplace=True)
train.dropna(subset=['price', 'bodyType'], inplace=True)
print(f'Records {train.shape[0]}, duplicates {train.duplicated().sum()}')

# statistics of NAN in train dataset
nulls = train.isnull().sum()
print('Number of empty features values in train dataset:')
nulls[nulls.values >0]

Records 111974, duplicates 0
Number of empty features values in train dataset:


complectation_dict          70
image                        2
modelDate               111974
model_info              111974
numberOfDoors               70
vehicleConfiguration    111974
vendor                  111974
Владельцы                    7
Владение                 80244
ПТС                          1
Состояние                   11
Таможня                     17
dtype: int64

In [15]:
# fill empty 'numberOfDoors' with value as the same bodyType has
doors = dict(train.groupby(['bodyType', 'numberOfDoors'])['numberOfDoors'].count().index)
train['numberOfDoors']=train.bodyType.map(doors)

# fill empty 'complectation_dict' with empty dict
train.complectation_dict = train.complectation_dict.apply(lambda q: dict() if pd.isna(q) else q)

In [16]:
train.numberOfDoors = train.numberOfDoors.apply(int)
train.parsing_unixtime = train.parsing_unixtime.apply(int)
train.productionDate = train.productionDate.apply(int)
train.sell_id = train.sell_id.apply(int)
train.mileage = train.mileage.apply(int)

In [17]:
nulls = train.isnull().sum()
print('Number of empty features values in train dataset:')
nulls[nulls.values >0]

Number of empty features values in train dataset:


image                        2
modelDate               111974
model_info              111974
vehicleConfiguration    111974
vendor                  111974
Владельцы                    7
Владение                 80244
ПТС                          1
Состояние                   11
Таможня                     17
dtype: int64

In [18]:
test.isnull().sum()

bodyType                    0
brand                       0
car_url                     0
color                       0
complectation_dict      28268
description                 0
engineDisplacement          0
enginePower                 0
equipment_dict           9996
fuelType                    0
image                       0
mileage                     0
modelDate                   0
model_info                  0
model_name                  0
name                        0
numberOfDoors               0
parsing_unixtime            0
priceCurrency               0
productionDate              0
sell_id                     0
super_gen                   0
vehicleConfiguration        0
vehicleTransmission         0
vendor                      0
Владельцы                   0
Владение                22691
ПТС                         1
Привод                      0
Руль                        0
Состояние                   0
Таможня                     0
dtype: int64

---
### 0.2. Checking 'test' and 'auto' datadset.
It is very important to understand when they was parsed. The prices in Russian is always run up,
so ML-model can be good to predict today prices, but several month ago parsed test dataset - no,
because only the another level of prices.

In [19]:
auto.price.mean(), train.price.mean() , train.price.mean()/auto.price.mean()

(1294586.3563303659, 1625792.069507207, 1.2558390265410155)

In [20]:
import time
print(time.ctime(test.parsing_unixtime.min()))
print(time.ctime(test.parsing_unixtime.max()))

Mon Oct 19 14:35:06 2020
Mon Oct 26 14:04:24 2020


### Resume:
test data was parsed nearly when was pased auto dataset - in september 2020.
Today's prises in my train dataset is higher on 25.584%.
So we must to divide current prices by the factor 1.255839.

In [21]:
train['price_current']=train.price
train.price = train.price / 1.255839

----
# 1. Clearin data


First of all I will normalize of filling data in train and test datasets, because some vectors have been download not in the same format

In [22]:
""" function to show filling difference between same vectors in train and test datasets"""
def vector_items(vector: str):
    """
    function to show filling difference between same vectors in train and test datasets
    :param vector: nave of vector
    :return: difference between train and test, test and train and unique values of test
    """
    train_set = set(train[vector])
    test_set = set(test[vector].unique())
    train_dif = train_set - test_set
    test_dif = test_set - train_set
    print(f'Statistics of items in vector: {vector}.\n ---------------------------')
    print(f'Train has {len(train_set)} unique items and has difference:\n{train_dif},')
    print('--')
    print(f'Test has {len(test_set)} unique items and has difference:\n{test_dif}.')
    print('Test_set values:')
    print(test_set)

## 1.2. bodyType

In [23]:
vector_items('bodyType')

Statistics of items in vector: bodyType.
 ---------------------------
Train has 23 unique items and has difference:
set(),
--
Test has 24 unique items and has difference:
{'фастбек'}.
Test_set values:
{'универсал 5 дв.', 'купе-хардтоп', 'кабриолет', 'лифтбек', 'седан 2 дв.', 'хэтчбек 3 дв.', 'микровэн', 'седан-хардтоп', 'пикап одинарная кабина', 'хэтчбек 5 дв.', 'пикап двойная кабина', 'внедорожник 5 дв.', 'пикап полуторная кабина', 'фургон', 'родстер', 'купе', 'седан', 'компактвэн', 'тарга', 'лимузин', 'фастбек', 'внедорожник открытый', 'внедорожник 3 дв.', 'минивэн'}


Body type of cars in train is wider then in test by value 'фастбек'. It is OK.

## 2.2. brand, model_name, model (new vector)

In [24]:
vector_items('brand')

Statistics of items in vector: brand.
 ---------------------------
Train has 12 unique items and has difference:
set(),
--
Test has 12 unique items and has difference:
set().
Test_set values:
{'TOYOTA', 'MERCEDES', 'VOLVO', 'AUDI', 'SKODA', 'NISSAN', 'LEXUS', 'VOLKSWAGEN', 'BMW', 'INFINITI', 'MITSUBISHI', 'HONDA'}


Brand - was a key for download, so vectors are equal.

In [25]:
train.model_name = train.model_name.str.upper()

In [26]:
train[train.model_name == 'S-КЛАСС']

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price,price_current
41563,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,"{'back-brake': 'Ventilated disc', 'feeding': '...",Авто новое пробег 2500км в самой топовой компл...,2.9 л,249 л.с.,"{'e-adjustment-wheel': True, 'multi-wheel': Tr...",Дизель,...,NaN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1.254142e+07,15750000.0
41564,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,"{'back-brake': 'Ventilated disc', 'feeding': '...",Автомобиль приобретён у официального дилера Па...,3.0 л,367 л.с.,"{'asr': True, 'roller-blind-for-rear-window': ...",Бензин,...,NaN,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1.262104e+07,15850000.0
41565,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,"{'back-brake': 'Ventilated disc', 'feeding': '...",ЛУКАВТО официальный дилер Mercedes Benz в Рос...,2.9 л,249 л.с.,"{'asr': True, 'roller-blind-for-rear-window': ...",Дизель,...,NaN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1.273969e+07,15999000.0
41566,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,"{'back-brake': 'Ventilated disc', 'feeding': '...",Внимание Только для клиентов AVILON Автомобили...,3.0 л,367 л.с.,"{'cruise-control': True, 'asr': True, 'roller-...",Бензин,...,NaN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1.272456e+07,15980000.0
41567,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,"{'back-brake': 'Ventilated disc', 'feeding': '...",В пленке самая полная комплектация нет только ...,3.0 л,367 л.с.,"{'asr': True, 'tinted-glass': True, 'roller-bl...",Бензин,...,NaN,1 владелец,8 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1.234235e+07,15500000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9545,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,синий,"{'back-brake': 'Ventilated disc', 'feeding': '...",Хороший автомобиль без нареканий требует мелки...,6.0 л,394 л.с.,{},Бензин,...,NaN,1 владелец,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен,3.025866e+05,380000.0
9546,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,синий,"{'back-brake': 'Disc', 'feeding': 'None', 'ful...",Нормальное состояние по кузову есть не критичн...,3.2 л,224 л.с.,{},Бензин,...,NaN,3 или более,NaN,Дубликат,задний,Левый,Не требует ремонта,Растаможен,2.946238e+05,370000.0
9547,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,"{'back-brake': 'Disc', 'feeding': 'None', 'ful...",Авто без проблемм Не шпаклеванный не из кусков...,4.3 л,279 л.с.,"{'cruise-control': True, 'engine-proof': True,...",Бензин,...,NaN,3 или более,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен,2.910843e+05,365555.0
9548,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,серый,"{'back-brake': 'Ventilated disc', 'feeding': '...",ЗАВОДСКАЯ БРОНЕКАПСУЛА Богатая комплектация Ра...,5.0 л,320 л.с.,{},Бензин,...,NaN,3 или более,NaN,Дубликат,задний,Левый,Не требует ремонта,Растаможен,2.906424e+05,365000.0


In [27]:
test[test.model_name == 'S_KLASSE']

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
15416,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,синий,NaN,Позвоните нам и получите дополнительную скидку...,5.5 LTR,388 N12,"{""electro-window-back"":true,""alloy-wheel-disks...",бензин,...,SEDAN AUTOMATIC 5.5,автоматическая,EUROPEAN,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
15449,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,NaN,В ДТП не участвовала.\n\n。Подушка безопасности...,4.7 LTR,435 N12,"{""cruise-control"":true,""asr"":true,""tinted-glas...",бензин,...,SEDAN AUTOMATIC 4.7,автоматическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
15453,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,NaN,ЛОТ: 01200668\nFAVORIT MOTORS KIA Юг\n\nВы мож...,3.0 LTR,235 N12,"{""asr"":true,""tinted-glass"":true,""esp"":true,""ad...",дизель,...,SEDAN AUTOMATIC 3.0,автоматическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
15492,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,"{""id"":""21016733"",""name"":""S 450 4MATIC"",""availa...",МБ-Беляево — официальный дилер «Мерседес-Бенц»...,3.0 LTR,367 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,...,SEDAN AUTOMATIC 3.0,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
15497,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,NaN,Автодилер года 2020\n\nПобедитель в номинации ...,4.7 LTR,435 N12,"{""cruise-control"":true,""airbag-rear-side"":true...",бензин,...,SEDAN AUTOMATIC 4.7,автоматическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34214,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,NaN,Автомобиль в отличном состоянии.Обслуживался в...,5.0 LTR,306 N12,NaN,бензин,...,SEDAN AUTOMATIC 5.0,автоматическая,EUROPEAN,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
34226,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,NaN,Автомобиль Приобретался у официального дилера ...,4.0 LTR,469 N12,NaN,бензин,...,SEDAN AUTOMATIC 4.0,автоматическая,EUROPEAN,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
34239,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,NaN,"Продаётся легендарный S500 короткая база, полн...",5.0 LTR,306 N12,NaN,бензин,...,SEDAN AUTOMATIC 5.0,автоматическая,EUROPEAN,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
34241,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,NaN,Автомобиль продается официальным дилером Автом...,3.0 LTR,235 N12,"{""cruise-control"":true,""asr"":true,""tinted-glas...",дизель,...,SEDAN AUTOMATIC 3.0,автоматическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен


In [28]:
vector_items('model_name')

Statistics of items in vector: model_name.
 ---------------------------
Train has 532 unique items and has difference:
{'A-КЛАСС', 'SLC', 'MARK II', 'V90 CROSS COUNTRY', 'C-HR', '3 СЕРИИ', 'X5 M', 'CORONA EXIV', 'SCEPTER', 'X-TRAIL', 'HILUX SURF', 'ALMERA CLASSIC', 'V40 CROSS COUNTRY', 'CARAVAN', 'CL-КЛАСС AMG', 'GOLF GTI', 'GL-КЛАСС', 'MEGA CRUISER', 'MOBILIO SPIKE', 'MASTERACE SURF', 'GLA AMG', 'YARIS VERSO', '190 (W201)', 'CARINA E', '2 СЕРИИ GRAN TOURER', 'RS 5', 'I-MIEV', 'M-КЛАСС AMG', '190 SL', '6 СЕРИИ', 'CLK-КЛАСС', 'SLK-КЛАСС', 'DELICA D:2', 'RS 3', 'CLS', 'RS 4', 'COROLLA VERSO', 'STEPWGN', 'GLS', 'R-КЛАСС', 'FAIRLADY Z', 'GL-КЛАСС AMG', 'Z3 M', 'G', 'DELICA D:5', 'RS 7', 'SPACE GEAR', 'RS Q3', 'SL-КЛАСС AMG', 'GLC COUPE', 'CR-V', 'CARINA ED', 'CLA', 'PASSAT CC', 'CIVIC FERIO', 'S-КЛАСС AMG', 'S-КЛАСС', 'X-КЛАСС', 'SPACE STAR', 'CLA AMG', 'GLE COUPE', 'COROLLA RUMION', 'E-КЛАСС AMG', 'ECLIPSE CROSS', 'HR-V', 'CLS AMG', 'COROLLA SPACIO', 'G-КЛАСС AMG', 'GLA', 'FR-V', 'GOLF PL

Model name has different filling format. Example is above. I prefer to make equal names from urls, because they are didn't mutate in time, and delete model name from analise.

In [29]:
# example: https://auto.ru/cars/used/sale/audi/a4/1106130217-e5fd99e4/
# /audi/a4 - brand and model
train['model'] = train.car_url.apply(lambda q: q.split('/')[7])
test['model'] = test.car_url.apply(lambda q: q.split('/')[7])

In [30]:
vector_items('model')

Statistics of items in vector: model.
 ---------------------------
Train has 532 unique items and has difference:
set(),
--
Test has 544 unique items and has difference:
{'simplex', 'e3', 'popular', '120', 'origin', 'i', '140', 'golf_r32', '100_series', '280zx', 'sera', 'lm'}.
Test_set values:
{'granvia', 'sl_klasse_amg', 'x3_m', 'passat_cc', 'partner', 'juke_nismo', 'spade', 'vista', 'armada', 'auris', 'vellfire', 'colt', 'fortuner', 'q30', 'esquire', 'transporter', '3er', 'grand_hiace', 'prius_alpha', 'prelude', 'w188', 'polo', 'outlander', 'largo', 'odyssey_na', 'v60_cross_country', 'pilot', 'odyssey', 'w123', 'presea', 'aqua', 'xc40', 'sienna', 'pathfinder', 'r_klasse', 'gs', 'taro', 'avancier', 'vezel', 's7', 'tts', 'w111', 'dayz_roox', 'libero', 'wingroad', 'patrol', 'rvr', 'x2', 'leaf', 'v_klasse', 'passo', 'dingo', 'insight', 'qx50', 'm5', 'w186', 'fuga', '2grandtourer', 's70', 'octavia', 'prairie', 'succeed', 'mega_cruiser', 'gl_klasse', 'corolla_levin', 'grandis', '460', 'cls

---
### resume:
Vectors 'model_name' and 'name' is very garbage. May be it will be worth to drop them and to use only
vectors 'brand' and 'model'


---
## 2.3. color

In [31]:
vector_items('color')

Statistics of items in vector: color.
 ---------------------------
Train has 16 unique items and has difference:
set(),
--
Test has 16 unique items and has difference:
set().
Test_set values:
{'синий', 'красный', 'коричневый', 'жёлтый', 'фиолетовый', 'серебристый', 'пурпурный', 'розовый', 'золотистый', 'зелёный', 'чёрный', 'белый', 'серый', 'оранжевый', 'бежевый', 'голубой'}


The color - is OK.

---
## 2.4. engineDisplacement, fuelType

Resume for following code:

1. Rename some long values in train dataset. May by it is worth to delete them, because in test dataset we haven't the same car fuel type?
2. Electro-car has no engine displacement. This vectors in train and test datasets filled in different ways. So I need to make some new vector with engine fuel-type-capacity for classification in one vector. For electro-cars - capacity will by power in kVt.
3. For ML I transform vector 'enginePower', leaving only power in horse value in numeric format.
4. In vector engineDisplacement reoplace english tail 'LTR' for russian 'л'.

In [32]:
vector_items('fuelType')

Statistics of items in vector: fuelType.
 ---------------------------
Train has 9 unique items and has difference:
{'Газ, газобаллонное оборудование', 'Бензин', 'Электро', 'Дизель', 'Дизель, газобаллонное оборудование', 'Гибрид, газобаллонное оборудование', 'Газ', 'Бензин, газобаллонное оборудование', 'Гибрид'},
--
Test has 5 unique items and has difference:
{'бензин', 'электро', 'гибрид', 'дизель', 'газ'}.
Test_set values:
{'бензин', 'электро', 'гибрид', 'дизель', 'газ'}


In [33]:
# replace long name ', газобаллонное оборудование' on '-гбо' and make all letters lower case
train.fuelType = train.fuelType.str.replace(', газобаллонное оборудование', '-гбо')
train.fuelType = train.fuelType.str.lower()

In [34]:
vector_items('fuelType')

Statistics of items in vector: fuelType.
 ---------------------------
Train has 9 unique items and has difference:
{'бензин-гбо', 'газ-гбо', 'дизель-гбо', 'гибрид-гбо'},
--
Test has 5 unique items and has difference:
set().
Test_set values:
{'бензин', 'электро', 'гибрид', 'дизель', 'газ'}


In [35]:
""" function to take only numeric data from string"""
def find_number(field: str):
    """
    :param field: string to find number
    :return: number (in string format)
    """

    # mask for search
    p = '[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+'

    # main cycle for searching
    if re.search(p, field) is not None:
        for catch in re.finditer(p, field):
            return catch[0]

In fact, electro-cars have no engine displacement.
In datasets vector 'engineDisplacement' electro-cars records have different filling.
In test dataset vector 'engineDisplacement' filled with *'LTR'*, in train dataset - they filled with *horse power value*.
in test dataset vector 'enginePower' filled with *horse power value*, in train dataset - with *kVt power value*

We need to make something uniform. So we will make new vector 'engineName' as mask ['fuelType'+' '+'engineDisplacement'] for non-electro and ['fuelType'+' '+ kVt power value] for electro cars.

In [36]:
# make the dict from pairs (horse power, kVt power)
kvt=dict()

qq = train[train.fuelType == 'электро'].groupby(['engineDisplacement', 'enginePower'])['enginePower'].count()
qq = qq.to_dict()
for item, value in qq.keys():
    kvt[int(find_number(item))] = int(find_number(value))
kvt

{109: 80,
 115: 85,
 136: 100,
 150: 110,
 154: 113,
 170: 125,
 179: 132,
 184: 135,
 218: 160,
 313: 230,
 408: 300,
 41: 30,
 64: 47,
 67: 49,
 74: 100}

In [37]:
""" function to make new uniform vector of type of engine as mentioned above"""
def engineDisplacement(record):
    """
    :param record: raw from dataset
    :return: value for new vector
    """

    # case of gasoline and diesel
    if record.fuelType != 'электро':
        result = record.fuelType + ' ' + find_number(record.engineDisplacement)

    # case of electro
    else:
        # case for train dataset, where we have filled vector engineDisplacement, so we can take kVt power direct
        if find_number(record.engineDisplacement):
            result = record.fuelType + ' ' + find_number(record.enginePower)

        # case for test dataset, where we only have horse power filling
        else:
            value_kvt = int(record.enginePower.split()[0])
            try:
                # use dict to find value of kVt power from horse power
                value_kvt = str(kvt[value_kvt])

            # someone we don't have value of kVt power, so we leave just 'электро'
            except:
                value_kvt = ''

            result = record.fuelType + ' ' + str(value_kvt)

    return result

In [38]:
train['engineName'] = train.apply(engineDisplacement, axis=1)
test['engineName'] = test.apply(engineDisplacement, axis=1)

In [39]:
vector_items('engineName')

Statistics of items in vector: engineName.
 ---------------------------
Train has 158 unique items and has difference:
{'бензин-гбо 2.7', 'бензин-гбо 4.7', 'электро 100', 'бензин-гбо 2.3', 'дизель-гбо 3.0', 'дизель-гбо 4.5', 'бензин-гбо 1.6', 'электро 47', 'электро 113', 'бензин-гбо 4.2', 'бензин-гбо 3.5', 'бензин-гбо 3.3', 'бензин-гбо 6.0', 'электро 160', 'бензин-гбо 3.6', 'электро 135', 'газ 1.8', 'бензин-гбо 4.8', 'дизель 3.3', 'бензин-гбо 4.4', 'бензин-гбо 1.5', 'бензин-гбо 3.7', 'газ-гбо 2.0', 'электро 230', 'бензин-гбо 3.4', 'бензин-гбо 2.9', 'бензин-гбо 3.0', 'гибрид-гбо 3.3', 'бензин-гбо 4.3', 'бензин-гбо 1.4', 'бензин-гбо 1.8', 'бензин-гбо 5.6', 'гибрид-гбо 3.5', 'бензин-гбо 1.3', 'бензин-гбо 4.6', 'бензин-гбо 1.7', 'дизель-гбо 2.8', 'бензин-гбо 5.5', 'бензин-гбо 2.0', 'бензин-гбо 4.5', 'гибрид-гбо 1.3', 'дизель 1.8', 'газ-гбо 1.4', 'бензин-гбо 4.0', 'газ-гбо 1.8', 'бензин-гбо 3.8', 'бензин-гбо 2.5', 'электро 30', 'бензин-гбо 5.7', 'бензин-гбо 2.8', 'бензин-гбо 2.4', 'бензин-г

In [40]:
test.enginePower = test.enginePower.str.replace('N12', 'л.с.')

In [41]:
train['enginePowerValue'] = train.enginePower
test['enginePowerValue'] = test.enginePower

In [42]:
def enginePowerExchange(record):
    if record.enginePower.split()[1] == 'кВт':
        result = find_number(record.engineDisplacement)
    else:
        result = find_number(record.enginePower)
    return result

In [43]:
train.enginePower = train.apply(enginePowerExchange, axis=1)
test.enginePower = test.apply(enginePowerExchange, axis=1)

In [44]:
train.enginePower = train.enginePower.apply(int)
test.enginePower = test.enginePower.apply(int)

In [45]:
vector_items('enginePower')

Statistics of items in vector: enginePower.
 ---------------------------
Train has 329 unique items and has difference:
{134, 274, 537, 410, 670, 415, 161, 34, 417, 41, 299, 176, 564, 314, 187, 700, 448, 449, 456, 206, 344, 89, 237, 503},
--
Test has 315 unique items and has difference:
{32, 195, 550, 487, 457, 42, 44, 430, 214, 30}.
Test_set values:
{30, 32, 38, 40, 42, 44, 46, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 60, 61, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 120, 121, 122, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 135, 136, 137, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 154, 155, 156, 157, 158, 159, 160, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 173, 174, 175, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 188, 189, 190, 192, 193, 194, 195, 197,

In [46]:
vector_items('engineDisplacement')

Statistics of items in vector: engineDisplacement.
 ---------------------------
Train has 68 unique items and has difference:
{'5.9 л', '179 л.с.', '115 л.с.', '1.7 л', '3.3 л', '313 л.с.', '2.3 л', '4.4 л', '6.6 л', '408 л.с.', '2.7 л', '4.0 л', '218 л.с.', '3.4 л', '1.3 л', '5.4 л', '1.4 л', '2.0 л', '2.5 л', '4.8 л', '1.9 л', '2.6 л', '3.8 л', '136 л.с.', '3.2 л', '5.7 л', '4.7 л', '1.8 л', '5.2 л', '2.1 л', '0.7 л', '5.6 л', '1.6 л', '3.7 л', '4.6 л', '109 л.с.', '64 л.с.', '154 л.с.', '2.4 л', '1.5 л', '67 л.с.', '184 л.с.', '2.9 л', '3.6 л', '6.3 л', '41 л.с.', '5.8 л', '3.5 л', '3.1 л', '1.1 л', '1.0 л', '3.0 л', '6.0 л', '5.0 л', '170 л.с.', '4.5 л', '4.3 л', '6.2 л', '150 л.с.', '4.2 л', '5.5 л', '3.9 л', '4.9 л', '1.2 л', '4.1 л', '2.8 л', '2.2 л', '74 л.с.'},
--
Test has 55 unique items and has difference:
{'5.7 LTR', '2.2 LTR', '3.6 LTR', '5.2 LTR', '4.7 LTR', '4.3 LTR', '1.2 LTR', '5.0 LTR', '0.7 LTR', '3.9 LTR', '1.9 LTR', '1.7 LTR', '3.4 LTR', '4.0 LTR', '2.6 LTR', '2.5 

In [47]:
test.engineDisplacement = test.engineDisplacement.str.replace('LTR', 'л')

In [48]:
vector_items('engineDisplacement')

Statistics of items in vector: engineDisplacement.
 ---------------------------
Train has 68 unique items and has difference:
{'109 л.с.', '179 л.с.', '115 л.с.', '136 л.с.', '64 л.с.', '154 л.с.', '313 л.с.', '67 л.с.', '170 л.с.', '150 л.с.', '184 л.с.', '41 л.с.', '408 л.с.', '218 л.с.', '74 л.с.'},
--
Test has 55 unique items and has difference:
{'5.3 л', ' л'}.
Test_set values:
{'5.9 л', '1.7 л', '3.3 л', '2.3 л', '4.4 л', '6.6 л', '2.7 л', '4.0 л', '3.4 л', '1.3 л', '5.4 л', '1.4 л', '2.0 л', '2.5 л', '4.8 л', '1.9 л', '2.6 л', '3.8 л', '3.2 л', '5.7 л', '5.3 л', '4.7 л', '1.8 л', ' л', '5.2 л', '2.1 л', '0.7 л', '5.6 л', '1.6 л', '3.7 л', '4.6 л', '1.5 л', '2.4 л', '2.9 л', '3.6 л', '6.3 л', '5.8 л', '3.5 л', '3.1 л', '1.1 л', '1.0 л', '3.0 л', '6.0 л', '5.0 л', '4.5 л', '6.2 л', '4.3 л', '4.2 л', '5.5 л', '4.9 л', '3.9 л', '1.2 л', '4.1 л', '2.8 л', '2.2 л'}


---
## 2.5. vehicleConfiguration

Vehicle configuration in train dataset has to be combined from data of body type, vehicle transmission and engine displacement

In [49]:
vector_items('vehicleTransmission')

Statistics of items in vector: vehicleTransmission.
 ---------------------------
Train has 4 unique items and has difference:
set(),
--
Test has 4 unique items and has difference:
set().
Test_set values:
{'механическая', 'роботизированная', 'автоматическая', 'вариатор'}


In [50]:
transmission = {
    'механическая': 'MECHANICAL',
    'вариатор': 'VARIATOR',
    'роботизированная': 'ROBOT',
    'автоматическая': 'AUTOMATIC'
}

In [51]:
temp_df = test[
    ['bodyType', 'vehicleConfiguration']].groupby(
    ['bodyType', 'vehicleConfiguration'])['vehicleConfiguration'].count().to_dict()

bodyTypeEng = dict()
for key, value in temp_df.keys():
    bodyTypeEng[key] = value.split()[0]

bodyTypeEng

{'внедорожник 3 дв.': 'ALLROAD_3_DOORS',
 'внедорожник 5 дв.': 'ALLROAD_5_DOORS',
 'внедорожник открытый': 'ALLROAD_OPEN',
 'кабриолет': 'CABRIO',
 'компактвэн': 'COMPACTVAN',
 'купе': 'COUPE',
 'купе-хардтоп': 'COUPE_HARDTOP',
 'лимузин': 'LIMOUSINE',
 'лифтбек': 'LIFTBACK',
 'микровэн': 'MICROVAN',
 'минивэн': 'MINIVAN',
 'пикап двойная кабина': 'PICKUP_TWO',
 'пикап одинарная кабина': 'PICKUP_ONE',
 'пикап полуторная кабина': 'PICKUP_ONE_HALF',
 'родстер': 'ROADSTER',
 'седан': 'SEDAN',
 'седан 2 дв.': 'SEDAN_2_DOORS',
 'седан-хардтоп': 'SEDAN_HARDTOP',
 'тарга': 'TARGA',
 'универсал 5 дв.': 'WAGON_5_DOORS',
 'фастбек': 'FASTBACK',
 'фургон': 'VAN',
 'хэтчбек 3 дв.': 'HATCHBACK_3_DOORS',
 'хэтчбек 5 дв.': 'HATCHBACK_5_DOORS'}

In [52]:
""" function to construct filling for vector 'vehicleConfiguration' from several another vectors"""
def vehicleConfiguration(record):
    """
    :param record: record from dataset (call by apply)
    :return: filling for field in vector with mask like : 'COUPE ROBOT 2.5'
    """
    body = bodyTypeEng[record.bodyType]
    trans = transmission[record.vehicleTransmission]
    disp = find_number(record.engineDisplacement)

    result = body + ' ' + trans
    if disp:
        result = result + ' ' + disp

    return result

In [53]:
train.vehicleConfiguration = train.apply(vehicleConfiguration, axis=1)

In [54]:
vector_items('vehicleConfiguration')

Statistics of items in vector: vehicleConfiguration.
 ---------------------------
Train has 764 unique items and has difference:
{'PICKUP_TWO AUTOMATIC 3.3', 'COUPE_HARDTOP AUTOMATIC 2.2', 'ALLROAD_3_DOORS AUTOMATIC 5.0', 'HATCHBACK_5_DOORS AUTOMATIC 67', 'COUPE VARIATOR 2.7', 'VAN AUTOMATIC 0.7', 'COUPE ROBOT 1.8', 'CABRIO VARIATOR 3.2', 'COUPE AUTOMATIC 4.5', 'ALLROAD_5_DOORS MECHANICAL 4.7', 'CABRIO MECHANICAL 1.5', 'ALLROAD_3_DOORS MECHANICAL 3.3', 'PICKUP_ONE MECHANICAL 1.6', 'COUPE_HARDTOP AUTOMATIC 3.0', 'WAGON_5_DOORS VARIATOR 1.3', 'VAN AUTOMATIC 3.0', 'SEDAN_2_DOORS MECHANICAL 1.3', 'COUPE AUTOMATIC 1.4', 'COUPE MECHANICAL 2.7', 'ALLROAD_3_DOORS MECHANICAL 0.7', 'SEDAN_HARDTOP AUTOMATIC 1.5', 'COUPE MECHANICAL 1.9', 'PICKUP_ONE MECHANICAL 1.7', 'CABRIO AUTOMATIC 2.2', 'WAGON_5_DOORS MECHANICAL 1.0', 'COUPE_HARDTOP MECHANICAL 2.0', 'MINIVAN AUTOMATIC 5.5', 'PICKUP_TWO MECHANICAL 4.0', 'PICKUP_ONE AUTOMATIC 5.7', 'ROADSTER ROBOT 1.8', 'SEDAN AUTOMATIC 109', 'CABRIO MECHANICAL 2

In [55]:
train.drop(['modelDate', 'model_info', 'model_name', 'name', 'price_current'], axis=1, inplace=True)
test.drop(['modelDate', 'model_info', 'model_name', 'name'], axis=1, inplace=True)

---
## 2.6. vendor

Vendor in test dataset has only two ubique value. So I fill train dataset with dictionary brand-vendor.

In [56]:
# prepare to filling vector 'vendor' in train dataset.
# make dictionary of brand-vendors
vend = test.groupby(['brand', 'vendor'])['vendor'].count().to_dict().keys()
vend=dict(vend)

In [57]:
train.vendor = train.brand.map(vend)

In [58]:
vector_items('vendor')

Statistics of items in vector: vendor.
 ---------------------------
Train has 2 unique items and has difference:
set(),
--
Test has 2 unique items and has difference:
set().
Test_set values:
{'EUROPEAN', 'JAPANESE'}


---
## 2.7. Владение, Владельцы

Transform vector 'Владельцы' to numeric value, delete vector 'Владение' - because it has a lot of NA.

In [59]:
vector_items('Владельцы')

Statistics of items in vector: Владельцы.
 ---------------------------
Train has 4 unique items and has difference:
{nan, '1 владелец', '2 владельца'},
--
Test has 3 unique items and has difference:
{'1\xa0владелец', '2\xa0владельца'}.
Test_set values:
{'1\xa0владелец', '3 или более', '2\xa0владельца'}


In [60]:
train['Владельцы'].fillna('3 или более', inplace=True)

In [61]:
test['owners'] = test['Владельцы']
test['Владельцы'] = test['Владельцы'].apply(find_number)

In [62]:
train['owners'] = train['Владельцы']
train['Владельцы'] = train['Владельцы'].apply(find_number)

In [63]:
train['Владельцы'] = train['Владельцы'].apply(int)
test['Владельцы'] = test['Владельцы'].apply(int)

In [64]:
vector_items('Владельцы')

Statistics of items in vector: Владельцы.
 ---------------------------
Train has 3 unique items and has difference:
set(),
--
Test has 3 unique items and has difference:
set().
Test_set values:
{1, 2, 3}


In [65]:
train.drop(['Владение'], axis=1, inplace=True)
test.drop(['Владение'], axis=1, inplace=True)

---
## 2.8. ПТС

Filling NA with 'Дубликат'

In [66]:
test['ПТС'].value_counts()

Оригинал    30098
Дубликат     4587
Name: ПТС, dtype: int64

In [67]:
train['ПТС'].fillna('Дубликат', inplace=True)
test['ПТС'].fillna('Дубликат', inplace=True)

In [68]:
vector_items('ПТС')

Statistics of items in vector: ПТС.
 ---------------------------
Train has 2 unique items and has difference:
set(),
--
Test has 2 unique items and has difference:
set().
Test_set values:
{'Дубликат', 'Оригинал'}


---
## 2.9. Состояние

In train dataset there are some records with NA 'Состояние'. Analysis of the records led to the conclusion that these options relate to car accidents. So I delete this records from dataset.
After that - delete vector as unuseful for ML

In [69]:
vector_items('Состояние')

Statistics of items in vector: Состояние.
 ---------------------------
Train has 2 unique items and has difference:
{nan},
--
Test has 1 unique items and has difference:
set().
Test_set values:
{'Не требует ремонта'}


In [70]:
train[train['Состояние'] != 'Не требует ремонта']

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,ПТС,Привод,Руль,Состояние,Таможня,price,model,engineName,enginePowerValue,owners
36691,внедорожник 5 дв.,MERCEDES,https://auto.ru/cars/used/sale/mercedes/glb_kl...,белый,"{'back-brake': 'Disc', 'feeding': 'Turbocharge...",Куплен 25 декабря 2020 у официального дилера в...,1.3 л,150,{},бензин,...,Оригинал,передний,Левый,NaN,Растаможен,2.134032e+06,glb_klasse,бензин 1.3,150 л.с.,1 владелец
67936,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,серебристый,"{'back-brake': 'Disc', 'feeding': 'None', 'ful...",Битая Цена ремонта на 150 тр,1.6 л,102,"{'cruise-control': True, 'tinted-glass': True,...",бензин,...,Оригинал,передний,Левый,NaN,Растаможен,1.592561e+05,octavia,бензин 1.6,102 л.с.,3 или более
77393,седан,TOYOTA,https://auto.ru/cars/used/sale/toyota/corolla/...,серый,"{'feeding': 'None', 'horse-power': 105, 'kvt-p...",Надо Делать ДВС Обмен Торг,1.6 л,105,{},бензин,...,Оригинал,передний,Правый,NaN,Растаможен,7.962804e+04,corolla,бензин 1.6,105 л.с.,2 владельца
77431,седан,TOYOTA,https://auto.ru/cars/used/sale/toyota/corolla/...,белый,"{'back-brake': 'Drum', 'feeding': 'None', 'ran...",продается легенда японского автопрома битая по...,1.6 л,115,{},бензин,...,Дубликат,передний,Правый,NaN,Растаможен,5.972103e+04,corolla,бензин 1.6,115 л.с.,3 или более
89883,хэтчбек 5 дв.,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/golf...,красный,"{'back-brake': 'Drum', 'feeding': 'None', 'ful...",Проблемы с электрикой производилась замена вых...,1.6 л,75,"{'ptf': True, 'abs': True, 'eco-leather': True...",бензин,...,Дубликат,передний,Левый,NaN,Растаможен,1.194421e+05,golf,бензин 1.6,75 л.с.,3 или более
89888,хэтчбек 5 дв.,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/golf...,чёрный,"{'back-brake': 'Drum', 'feeding': 'None', 'ful...",Замена порогов задняя юбка днище обработана ан...,1.6 л,72,{},бензин,...,Оригинал,передний,Левый,NaN,Растаможен,1.274049e+05,golf,бензин 1.6,72 л.с.,3 или более
94015,универсал 5 дв.,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/pass...,белый,"{'back-brake': 'Drum', 'feeding': 'None', 'ful...",На разбор или под восстановление без АКБ Торг ...,1.8 л,107,{},бензин,...,Оригинал,передний,Левый,NaN,Растаможен,5.175823e+04,passat,бензин 1.8,107 л.с.,3 или более
16755,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/rapid/110...,серый,"{'back-brake': 'Drum', 'feeding': 'None', 'ful...",Официальный дилерский центр РОЛЬФ Премиум Вешк...,1.6 л,110,"{'airbag-driver': True, 'rain-sensor': True, '...",бензин,...,Оригинал,передний,Левый,NaN,Растаможен,6.131359e+05,rapid,бензин 1.6,110 л.с.,1 владелец
18626,хэтчбек 5 дв.,TOYOTA,https://auto.ru/cars/used/sale/toyota/corolla/...,белый,"{'back-brake': 'Disc', 'feeding': 'None', 'ful...",Машина была привезена в Россию из Германии ещё...,1.6 л,110,{},бензин,...,Оригинал,передний,Левый,NaN,Растаможен,1.672189e+05,corolla,бензин 1.6,110 л.с.,2 владельца
21554,седан,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/vent...,серый,"{'back-brake': 'Drum', 'feeding': 'None', 'ful...",Битый двигатель не пострадал система охлаждени...,1.8 л,75,{},бензин,...,Оригинал,передний,Левый,NaN,Растаможен,7.962804e+04,vento,бензин 1.8,75 л.с.,3 или более


In [71]:
train.dropna(subset=['Состояние'], inplace=True)

In [72]:
vector_items('Состояние')

Statistics of items in vector: Состояние.
 ---------------------------
Train has 1 unique items and has difference:
set(),
--
Test has 1 unique items and has difference:
set().
Test_set values:
{'Не требует ремонта'}


In [73]:
train.drop(['Состояние'], axis=1, inplace=True)
test.drop(['Состояние'], axis=1, inplace=True)

---
## 2.10 Таможня

The same history with vector 'Таможня'. Analysis of the records led to the conclusion that these options relate to non-cleared cars or dubious ownership or purchase histories. So I delete this records from dataset.
After that - delete vector as unuseful for ML

In [74]:
vector_items('Таможня')

Statistics of items in vector: Таможня.
 ---------------------------
Train has 2 unique items and has difference:
{nan},
--
Test has 1 unique items and has difference:
set().
Test_set values:
{'Растаможен'}


In [75]:
train.dropna(subset=['Таможня'], inplace=True)

In [76]:
vector_items('Таможня')

Statistics of items in vector: Таможня.
 ---------------------------
Train has 1 unique items and has difference:
set(),
--
Test has 1 unique items and has difference:
set().
Test_set values:
{'Растаможен'}


In [77]:
train.drop(['Таможня'], axis=1, inplace=True)
test.drop(['Таможня'], axis=1, inplace=True)

---
## 2.11. climate (new vector from equpment data)

In [78]:
""" function to make filling fo vector 'climate'"""
def climate(record):
    """
    :param record: record from dataset called by apply
    :return: data for vector 'climate'
    """

    text = record.equipment_dict
    result='no data'

    try:
        text.index('condition')
        result = 'condition'
    except:
        pass

    try:
        text.index('climate-control')
        result = 'climate-control'
    except:
        pass

    return result

In [79]:
train['climate'] = train.apply(climate, axis=1)
test['climate'] = test.apply(climate, axis=1)

---
## 2.0 Prepare for Feature engineering

In [80]:
# delete some vectors
train.drop(['complectation_dict', 'equipment_dict', 'car_url',
            'image', 'description', 'priceCurrency'], axis=1, inplace=True)
test.drop(['complectation_dict', 'equipment_dict', 'car_url',
           'image', 'description', 'priceCurrency'], axis=1, inplace=True)

In [81]:
# append test dataset to train dataset with value of price=0
# vector-mark of test - 'test'
test['price']=0
test['test']=1
train['test']=0
data = train.append(test)

In [82]:
data.shape, train.shape, test.shape

((146632, 26), (111946, 26), (34686, 26))

In [83]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146632 entries, 0 to 34685
Data columns (total 26 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   bodyType              146632 non-null  object 
 1   brand                 146632 non-null  object 
 2   color                 146632 non-null  object 
 3   engineDisplacement    146632 non-null  object 
 4   enginePower           146632 non-null  int64  
 5   fuelType              146632 non-null  object 
 6   mileage               146632 non-null  int64  
 7   numberOfDoors         146632 non-null  int64  
 8   parsing_unixtime      146632 non-null  int64  
 9   productionDate        146632 non-null  int64  
 10  sell_id               146632 non-null  int64  
 11  super_gen             146632 non-null  object 
 12  vehicleConfiguration  146632 non-null  object 
 13  vehicleTransmission   146632 non-null  object 
 14  vendor                146632 non-null  object 
 15  В

---
# 2.FE

IDEAS
* average mileage per year
* (X) average mileage by owner
* rarity
* older 3-5 year
* (X) color clustering
* body clustering
* car ownership period

(X) - no useful idea

---
## 2.1. average mileage per year, average mileage by owner

In [84]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146632 entries, 0 to 34685
Data columns (total 26 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   bodyType              146632 non-null  object 
 1   brand                 146632 non-null  object 
 2   color                 146632 non-null  object 
 3   engineDisplacement    146632 non-null  object 
 4   enginePower           146632 non-null  int64  
 5   fuelType              146632 non-null  object 
 6   mileage               146632 non-null  int64  
 7   numberOfDoors         146632 non-null  int64  
 8   parsing_unixtime      146632 non-null  int64  
 9   productionDate        146632 non-null  int64  
 10  sell_id               146632 non-null  int64  
 11  super_gen             146632 non-null  object 
 12  vehicleConfiguration  146632 non-null  object 
 13  vehicleTransmission   146632 non-null  object 
 14  vendor                146632 non-null  object 
 15  В

In [85]:
def mil_year(record):
    date = record.productionDate
    age = 2021 - date - record.test + 1
    result = record.mileage / age
    return result

In [86]:
data['mileage_year'] = data.apply(mil_year, axis=1)

In [93]:
data['mileage_owner'] = data.apply(lambda r: r.mileage / r['Владельцы'], axis=1)

---
## 2.2. color clustering

In [142]:
train.color.value_counts(normalize=True)

чёрный         0.287147
белый          0.226984
серый          0.135038
серебристый    0.110455
синий          0.086041
красный        0.042485
коричневый     0.036786
зелёный        0.023726
бежевый        0.015856
голубой        0.010407
пурпурный      0.006637
золотистый     0.006155
фиолетовый     0.005529
жёлтый         0.003412
оранжевый      0.003046
розовый        0.000295
Name: color, dtype: float64

In [153]:
np.around(train.groupby(['color'])['price'].agg(['mean', 'count']).sort_values(by='mean',ascending=False),0)

,mean,count
color,,
чёрный,1662717.0,32145
коричневый,1403319.0,4118
белый,1390148.0,25410
оранжевый,1360564.0,341
серый,1184607.0,15117
синий,1181653.0,9632
бежевый,955126.0,1775
красный,881731.0,4756
жёлтый,873158.0,382


In [163]:
# https://autoreview.ru/news/samye-populyarnye-cveta-avtomobiley-statistika-2020-goda
color_pop ={
     'бежевый': 15,
     'белый': 227,
     'голубой': 10,
     'жёлтый': 3,
     'зелёный': 23,
     'золотистый': 6,
     'коричневый': 36,
     'красный': 42,
     'оранжевый': 3,
     'пурпурный': 6,
     'розовый': 0,
     'серебристый': 110,
     'серый':135,
     'синий': 86,
     'фиолетовый': 5,
     'чёрный': 287
     }

In [ ]:
color_pop ={
    'бежевый': 1,
    'белый': 38,
    'голубой': 1,
    'жёлтый': 2,
    'зелёный': 1,
    'золотистый': 2,
    'коричневый': 3,
    'красный': 5,
    'оранжевый': 1,
    'пурпурный': 1,
    'розовый': 1,
    'серебристый': 9,
    'серый':15,
    'синий': 7,
    'фиолетовый': 1,
    'чёрный': 19
}

In [164]:
data['color_pop']=data.color.map(color_pop)

In [167]:
data.drop(['color_pop', 'mileage_owner'], axis=1, inplace=True)

---
# Making dataset with final set of features

In [87]:
data.to_csv(path+'data_eda_ksv.csv', index=False)

----

In [7]:
#pip install category_encoders

Note: you may need to restart the kernel to use updated packages.


In [6]:
#import category_encoders as ce

In [133]:
""" function for checking feature importance"""
def feature_imp(dataset, regressor, encoder, test_drop = True, target=target):
    """
    function for checking feature importance
    :param dataset: dataset for ML
    :param regressor: algorithm of ML
    :param encoder: encoder for categorical vectors
    :param test_drop: boolean flag whether need to separate test data from dataset
    :param target: name of target
    :return: list of features with weights descending, score of algorithm and MAPE score
    """

    # copy dataset useful if you need to drop some vectors from dataset
    dataset=dataset.copy()

    # checkin boolean flag to drop test data from dataset
    if test_drop:
        dataset = dataset[dataset.test == 0]

    # drop vector 'test'
    dataset.drop(['test'], axis=1, inplace=True)

    # make variables x and y for ML model
    x, y = dataset.drop(target, axis=1), np.log(dataset[target])

    # encoding categorical vectors with encoder
    encoder.fit(x, y)
    X = encoder.transform(x)

    # split for validate and fit ML model
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=SEED)
    regressor.fit(X_train, y_train)

    # make dataframe with features importance
    importance = regressor.feature_importances_
    value = pd.DataFrame(data=importance, index=X.columns, columns=['importance'])
    print(value.importance.sort_values(ascending=False))

    # calculate ML model score
    score = regressor.score(X_test, y_test)
    print(f'Score of regressor {score*100:0.2f}%')

    # calculate MAPE score
    predict = np.exp(regressor.predict(X_test))
    print(f"MAPE: {(mape(np.exp(y_test), predict))*100:0.2f}%")

In [251]:
list_to_drop = ['ПТС', 'Владельцы', 'fuelType', 'climate'] #, 'numberOfDoors', 'vehicleTransmission', 'Руль', 'owners', ]
parameters = {
    'regressor': RandomForestRegressor(random_state=SEED),
    'encoder': ce.TargetEncoder()
}
feature_imp(data.drop(list_to_drop, axis=1), **parameters)
# 97,28%, MAPE 9,32%  RF Target  ['ПТС', 'Владельцы', 'fuelType', 'climate']


productionDate          0.550408
enginePowerValue        0.162881
model                   0.158358
enginePower             0.023451
mileage                 0.019268
parsing_unixtime        0.018238
engineDisplacement      0.017829
vehicleConfiguration    0.011065
engineName              0.007806
sell_id                 0.005292
mileage_year            0.005063
Привод                  0.004166
brand                   0.004143
bodyType                0.003828
color                   0.002006
super_gen               0.001146
owners                  0.001139
vendor                  0.001054
vehicleTransmission     0.000990
numberOfDoors           0.000989
Руль                    0.000879
Name: importance, dtype: float64
Score of regressor 97.28%
MAPE: 9.32%


In [252]:
parameters = {
    'regressor': CatBoostRegressor(iterations = 5000,
                                   random_seed = SEED,
                                   eval_metric='MAPE',
                                   custom_metric=['R2', 'MAE'],
                                   silent=True,
                                   ),
    'encoder': ce.CatBoostEncoder()
}
feature_imp(data, **parameters)

productionDate          39.725214
model                   16.765604
enginePower              7.812171
enginePowerValue         7.687615
mileage                  6.535942
vehicleConfiguration     3.582430
parsing_unixtime         2.600189
brand                    2.182875
engineName               2.044012
Привод                   2.009141
engineDisplacement       1.751512
bodyType                 1.591147
fuelType                 0.951279
vendor                   0.870902
mileage_year             0.736897
Руль                     0.581465
numberOfDoors            0.450105
sell_id                  0.417147
Владельцы                0.344988
vehicleTransmission      0.318904
super_gen                0.312482
climate                  0.270384
owners                   0.215072
color                    0.166500
ПТС                      0.076021
Name: importance, dtype: float64
Score of regressor 96.48%
MAPE: 12.32%


In [125]:
parameters = {
    'regressor': DecisionTreeRegressor(random_state=SEED),
    'encoder': ce.CatBoostEncoder()
}
feature_imp(data, **parameters)

productionDate          0.540784
enginePowerValue        0.183545
model                   0.155123
mileage                 0.022575
enginePower             0.020910
parsing_unixtime        0.019883
engineName              0.011431
engineDisplacement      0.008528
vehicleConfiguration    0.005710
sell_id                 0.004733
mileage_year            0.004348
brand                   0.003803
bodyType                0.003530
Привод                  0.003314
mileage_owner           0.002847
color                   0.001659
super_gen               0.001380
vehicleTransmission     0.001207
Руль                    0.001043
numberOfDoors           0.000946
vendor                  0.000866
fuelType                0.000597
ПТС                     0.000467
owners                  0.000461
Владельцы               0.000310
test                    0.000000
Name: importance, dtype: float64
Score of regressor 94.60%
MAPE: 13.41%


In [253]:
parameters = {
    'regressor': ExtraTreesRegressor(random_state=SEED),
    'encoder': ce.TargetEncoder()
}
feature_imp(data, **parameters)
# 96,64, MAPE 10,99%
# 96,65   MAPE 10,95%

productionDate          0.283181
model                   0.165267
enginePowerValue        0.107606
Привод                  0.100611
vehicleConfiguration    0.089767
mileage                 0.069859
enginePower             0.030351
owners                  0.025407
engineName              0.024936
engineDisplacement      0.016588
bodyType                0.011623
brand                   0.010854
Владельцы               0.010215
mileage_year            0.006935
parsing_unixtime        0.006662
vehicleTransmission     0.006288
climate                 0.005475
vendor                  0.005272
Руль                    0.005003
fuelType                0.004592
sell_id                 0.003838
color                   0.003334
numberOfDoors           0.003155
ПТС                     0.001630
super_gen               0.001552
Name: importance, dtype: float64
Score of regressor 96.65%
MAPE: 10.95%


In [127]:
parameters = {
    'regressor': LGBMRegressor(random_state=SEED),
    'encoder': ce.CatBoostEncoder()
}
feature_imp(data, **parameters)

model                   455
productionDate          434
parsing_unixtime        359
enginePower             300
brand                   181
enginePowerValue        165
mileage                 148
vehicleConfiguration    114
mileage_year             96
bodyType                 92
engineDisplacement       84
super_gen                75
engineName               73
fuelType                 64
Руль                     57
Привод                   56
sell_id                  55
numberOfDoors            44
mileage_owner            35
Владельцы                31
vehicleTransmission      30
vendor                   21
color                    20
ПТС                      11
owners                    0
test                      0
Name: importance, dtype: int32
Score of regressor 95.42%
MAPE: 14.38%


In [254]:
parameters = {
    'regressor': XGBRegressor(random_state=SEED),
    'encoder': ce.CatBoostEncoder()
}
feature_imp(data, **parameters)

productionDate          0.386466
enginePowerValue        0.166478
model                   0.109561
vehicleConfiguration    0.057470
engineDisplacement      0.052641
engineName              0.038182
Привод                  0.034812
enginePower             0.025553
mileage                 0.019464
bodyType                0.017913
Владельцы               0.014442
fuelType                0.012127
brand                   0.010410
Руль                    0.009078
vehicleTransmission     0.008931
numberOfDoors           0.007294
vendor                  0.007161
climate                 0.006069
parsing_unixtime        0.004321
super_gen               0.004097
mileage_year            0.002653
ПТС                     0.002559
sell_id                 0.001525
color                   0.000792
owners                  0.000000
Name: importance, dtype: float32
Score of regressor 96.30%
MAPE: 12.78%


----
# CatBosting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=SEED)

In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                          )
model.fit(X_train, y_train,
          #cat_features=cat_features_ids,
          eval_set=(X_test, y_test),
          verbose_eval=0,
          use_best_model=True,
          #plot=True
          )

model.save_model('catboost_single_model_baseline.model')

In [ ]:
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                          )
model.fit(X_train, np.log(y_train),
          #cat_features=cat_features_ids,
          eval_set=(X_test, np.log(y_test)),
          verbose_eval=0,
          use_best_model=True,
          #plot=True
          )

model.save_model('catboost_single_model_2_baseline.model')

In [ ]:
predict_test = np.exp(model.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

---
# Test various models

In [ ]:
test_classifires = {
    'RandomForest': RandomForestRegressor(n_estimators=300, random_state = SEED),
    'DecisionTree': DecisionTreeRegressor(random_state=SEED),
    'ExtraTrees'  : ExtraTreesRegressor(random_state=SEED),
    'LightGBM'    : LGBMRegressor(random_state=SEED),
    'XGBoost'     : XGBRegressor(random_state=SEED)
}

In [ ]:
results = dict()
for item in tqdm(test_classifires):
    cls=test_classifires[item]

    cls.fit(X_train, y_train)
    predict =cls.predict(X_test)

    cls.fit(X_train, np.log(y_train))
    predict_log = np.exp(cls.predict(X_test))

    results[item] = [round(mape(y_test, predict)*100,2), round(mape(y_test, predict_log)*100,2)]
results

---
# RandomForest best parameters

In [ ]:
# Making best parameters searching diapasons
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(start = 1, stop = 15, num = 15)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
min_samples_leaf = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
bootstrap = [True, False]
param_dist = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'bootstrap': bootstrap}

# Randomized search. Making 3 cross validation (cv) for each 100 iterations candidates
model_rfc = RandomForestRegressor(random_state = SEED)
rs = RandomizedSearchCV(model_rfc,
                        param_dist,
                        n_iter = 100,
                        cv = 3,
                        verbose = 1,
                        n_jobs=-1,
                        random_state=SEED)
rs.fit(X_train, np.log(y_train))
rfc_best = rs.best_params_
rfc_best['random_state']=SEED
rfc_best
# approx. 16 minutes
# {'n_estimators': 200,
# 'min_samples_split': 2,
# 'min_samples_leaf': 2,
# 'max_features': 'sqrt',
# 'max_depth': 14,
# 'bootstrap': False,
# 'random_state': 42}

In [ ]:
#
model_rfc = RandomForestRegressor(**rfc_best)

model_rfc.fit(X_train, np.log(y_train))
predict_log = np.exp(model_rfc.predict(X_test))
round(mape(y_test, predict_log)*100,2)

In [ ]:
rs_df = pd.DataFrame(rs.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
rs_df = rs_df.drop([
    'mean_fit_time',
    'std_fit_time',
    'mean_score_time',
    'std_score_time',
    'params',
    'split0_test_score',
    'split1_test_score',
    'split2_test_score',
    'std_test_score'],
    axis=1)

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=2)
sns.set(style="whitegrid", color_codes=True, font_scale = 2)
fig.set_size_inches(30,25)
sns.barplot(x='param_n_estimators', y='mean_test_score', data=rs_df, ax=axs[0,0], color='lightgrey')
axs[0,0].set_ylim([.83,.95])
axs[0,0].set_title(label = 'n_estimators', size=30, weight='bold')
sns.barplot(x='param_min_samples_split', y='mean_test_score', data=rs_df, ax=axs[0,1], color='coral')
axs[0,1].set_ylim([.85,.95])
axs[0,1].set_title(label = 'min_samples_split', size=30, weight='bold')
sns.barplot(x='param_min_samples_leaf', y='mean_test_score', data=rs_df, ax=axs[0,2], color='lightgreen')
axs[0,2].set_ylim([.80,.95])
axs[0,2].set_title(label = 'min_samples_leaf', size=30, weight='bold')
sns.barplot(x='param_max_features', y='mean_test_score', data=rs_df, ax=axs[1,0], color='wheat')
axs[1,0].set_ylim([.80,.95])
axs[1,0].set_title(label = 'max_features', size=30, weight='bold')
sns.barplot(x='param_max_depth', y='mean_test_score', data=rs_df, ax=axs[1,1], color='lightpink')
axs[1,1].set_ylim([.80,.95])
axs[1,1].set_title(label = 'max_depth', size=30, weight='bold')
sns.barplot(x='param_bootstrap',y='mean_test_score', data=rs_df, ax=axs[1,2], color='skyblue')
axs[1,2].set_ylim([.80,.95])
axs[1,2].set_title(label = 'bootstrap', size=30, weight='bold')
plt.show()

### resume:
unfortunately, no one 'best' parametrs did better score. Only the base settings gave the best.